In [2]:
import json
import pandas as pd
import numpy as np
import re
from datetime import datetime

In [3]:
file_path = "/Users/alyssanguyen/Desktop/IRLE_scraping/csv_files/final_ca_localff.csv"
ca_ff = pd.read_csv(file_path)
file_path_2 = "/Users/alyssanguyen/Desktop/IRLE_scraping/csv_files/uszips.csv"
ca_zip_count = pd.read_csv(file_path_2)
file_path_3 = "/Users/alyssanguyen/Desktop/IRLE_scraping/csv_files/processed_prices_ubereats_ca_ff_03222024.csv"
example = pd.read_csv(file_path_3)

In [4]:
#Drop all the columns we don't need 
ca_ff_ = ca_ff.drop(columns=['Unnamed: 0', 'inputted_location','restaurant_distance'])
ca_ff_ = ca_ff_.dropna(subset=['restaurant_location'])

In [5]:
ca_zip_count = ca_zip_count[['zip', 'county_name']]

In [6]:
ca_ff_

,Unnamed: 0.1,restaurant_name,menu_item,menu_item_price,restaurant_location,restaurant_rating,number_of_ratings
0,0,Burger Station,Veggie Delight Salad,13.45,"10361 Rockingham Drive, Sacramento, CA, 95827, US",0 mi,NaN
1,1,Burger Station,Chicken Caesar Salad,14.10,"10361 Rockingham Drive, Sacramento, CA, 95827, US",0 mi,NaN
2,2,Burger Station,Green Salad,12.15,"10361 Rockingham Drive, Sacramento, CA, 95827, US",0 mi,NaN
3,3,Burger Station,Chef's Salad,14.10,"10361 Rockingham Drive, Sacramento, CA, 95827, US",0 mi,NaN
4,4,Burger Station,Oriental Salad,14.10,"10361 Rockingham Drive, Sacramento, CA, 95827, US",0 mi,NaN
...,...,...,...,...,...,...,...
9461,9331,R Burgers,Buffalo Dipping Sauce,0.25,"2590 Geer Rd, Turlock, CA, 95382, US",4.3,62
9462,9332,R Burgers,Honey Mustard Dipping Sauce,0.25,"2590 Geer Rd, Turlock, CA, 95382, US",4.3,62
9463,9333,R Burgers,Ranch Dipping Sauce,0.25,"2590 Geer Rd, Turlock, CA, 95382, US",4.3,62
9464,9334,R Burgers,Sweet &amp; Sour Dipping Sauce,0.25,"2590 Geer Rd, Turlock, CA, 95382, US",4.3,62


In [7]:
#restaurant_rating cleaning 
# Ensure the column is of string type using .loc
ca_ff_.loc[:, 'restaurant_rating'] = ca_ff_['restaurant_rating'].astype(str)

# Count rows containing 'mi'
rows_with_mi = ca_ff_['restaurant_rating'].str.contains('mi').sum()
print("Number of rows with 'mi' in restaurant rating:", rows_with_mi)

# Replace invalid ratings ending with 'mi' with '0' using .loc
ca_ff_.loc[:, 'restaurant_rating'] = ca_ff_['restaurant_rating'].str.replace(r'.*mi$', '0', regex=True)

Number of rows with 'mi' in restaurant rating: 130


In [8]:
#converting data types 
ca_ff_['restaurant_name'] = ca_ff_['restaurant_name'].astype('string')
ca_ff_['menu_item'] = ca_ff_['menu_item'].astype('string')
ca_ff_['menu_item'] = ca_ff_['menu_item'].str.replace(r'\s+', ' ', regex=True)
ca_ff_['restaurant_location'] = ca_ff_['restaurant_location'].astype('string')
ca_ff_['restaurant_rating'] = ca_ff_['restaurant_rating'].str.strip().astype(float)

In [9]:
#cleaning up string columns 

ca_ff_['menu_item'] = ca_ff_['menu_item'].str.lower()
ca_ff_['restaurant_location'] = ca_ff_['restaurant_location'].str.lower()
ca_ff_['restaurant_name'] = ca_ff_['restaurant_name'].str.replace('_', ' ')

#remove special characters
ca_ff_['menu_item'] = ca_ff_['menu_item'].apply(lambda x: ''.join(ch for ch in x if ch.isalnum() or ch.isspace()))
ca_ff_#cleaning up string columns 

ca_ff_['menu_item'] = ca_ff_['menu_item'].str.lower()
ca_ff_['restaurant_location'] = ca_ff_['restaurant_location'].str.lower()
ca_ff_['restaurant_name'] = ca_ff_['restaurant_name'].str.replace('_', ' ')

#remove special characters
ca_ff_['menu_item'] = ca_ff_['menu_item'].apply(lambda x: ''.join(ch for ch in x if ch.isalnum() or ch.isspace()))
ca_ff_['restaurant_name'].unique()

<StringArray>
[      'Burger Station',         'Super Burger',    'Big Mouth Burgers',
        'P & G Burgers',        'Betty Burgers',        'Burger Basket',
          'Hook Burger',        'Burger Palace',      'Varsity Burgers',
       'Burger Factory', 'Cypress Best Burgers', 'Gold Star Hamburgers',
            'R Burgers',          'Burger Boss']
Length: 14, dtype: string

In [10]:
def price_list(x):
    return list(x)

def mean_non_zero(x):
    return np.mean(x[x != 0]) if np.any(x != 0) else 0

def median_non_zero(x):
    return np.median(x[x != 0]) if np.any(x != 0) else 0

def std_non_zero(x):
    return np.std(x[x != 0]) if np.any(x != 0) else 0

In [107]:
#"BJ'S"

ca_ff_bj = ca_ff_[ca_ff_['restaurant_name'] == 'B&J']

#First part of grouping 

agg_funcs = {
    'menu_item_price': [mean_non_zero, median_non_zero, std_non_zero],  # calculate the average, median, and standard dev PRICE
    'restaurant_rating': 'mean', # calculate the average RATING 
    'menu_item' : 'count',
    'number_of_ratings': 'first'
}

grouped_bj = ca_ff_bj.groupby(['restaurant_name','restaurant_location']).agg(agg_funcs).reset_index()
grouped_bj.columns = [' '.join(col).strip() for col in grouped_bj.columns.values]

In [108]:
#Second part of grouping 
bj_lst = ['burger','french fries']

# Filter rows where 'menu_item' contains any item in mcd_lst
menu_items_bj = ca_ff_bj[ca_ff_bj['menu_item'].isin(bj_lst)].sort_values('menu_item')
menu_items_bj = menu_items_bj.drop_duplicates(subset=['restaurant_name', 'restaurant_location', 'menu_item'])

grouped_bj_2 = menu_items_bj.groupby(['restaurant_name', 'restaurant_location'])['menu_item_price'].agg(price_list).reset_index()

grouped_bj_2[['hamburger','fries']] = grouped_bj_2['menu_item_price'].apply(pd.Series)
grouped_bj_2.drop(columns=['menu_item_price'], inplace=True)

#Merging the grouped dfs together 
merged_bj = pd.merge(grouped_bj, grouped_bj_2, on=['restaurant_name', 'restaurant_location'], how='inner')
merged_bj['combo'] = np.nan
merged_bj['specialty_item'] = np.nan
merged_bj['cheeseburger'] = np.nan

merged_bj

,restaurant_name,restaurant_location,menu_item_price mean_non_zero,menu_item_price median_non_zero,menu_item_price std_non_zero,restaurant_rating mean,menu_item count,number_of_ratings first,hamburger,fries,combo,specialty_item,cheeseburger
0,B&J,"6202 3rd st, san francisco, ca, 94124, us",7.45,8.0,2.647898,4.6,23,500+,8.5,6.0,NaN,NaN,NaN


In [11]:
#"Barney's Gourmet Hamburgers"

ca_ff_bm = ca_ff_[ca_ff_['restaurant_name'] == 'Big Mouth Burgers']

#First part of grouping 

agg_funcs = {
    'menu_item_price': [mean_non_zero, median_non_zero, std_non_zero],  # calculate the average, median, and standard dev PRICE
    'restaurant_rating': 'mean', # calculate the average RATING 
    'menu_item' : 'count',
    'number_of_ratings': 'first'
}

grouped_bm = ca_ff_bm.groupby(['restaurant_name','restaurant_location']).agg(agg_funcs).reset_index()
grouped_bm.columns = [' '.join(col).strip() for col in grouped_bm.columns.values]

In [12]:
#Second part of grouping 
bm_lst = ['big mouth burger', 'blue cheeseburger', 'french fries']

# Filter rows where 'menu_item' contains any item in mcd_lst
menu_items_bm = ca_ff_bm[ca_ff_bm['menu_item'].isin(bm_lst)].sort_values('menu_item')
menu_items_bm = menu_items_bm.drop_duplicates(subset=['restaurant_name', 'restaurant_location', 'menu_item'])

grouped_bm_2 = menu_items_bm.groupby(['restaurant_name', 'restaurant_location'])['menu_item_price'].agg(price_list).reset_index()

grouped_bm_2[['hamburger', 'cheeseburger', 'fries']] = grouped_bm_2['menu_item_price'].apply(pd.Series)
grouped_bm_2.drop(columns=['menu_item_price'], inplace=True)

#Merging the grouped dfs together 
merged_bm = pd.merge(grouped_bm, grouped_bm_2, on=['restaurant_name', 'restaurant_location'], how='inner')
merged_bm['combo'] = np.nan
merged_bm['specialty_item'] = np.nan


merged_bm

,restaurant_name,restaurant_location,menu_item_price mean_non_zero,menu_item_price median_non_zero,menu_item_price std_non_zero,restaurant_rating mean,menu_item count,number_of_ratings first,hamburger,cheeseburger,fries,combo,specialty_item
0,Big Mouth Burgers,"3392 24th st, san francisco, ca, 94110, us",13.464286,14.95,4.512658,4.6,70,500+,15.95,16.95,4.95,NaN,NaN
1,Big Mouth Burgers,"851 cherry ave, san bruno, ca, 94066, us",14.185714,15.95,4.366366,4.7,84,700+,15.95,16.95,5.25,NaN,NaN


In [13]:
#'Burger Station'

ca_ff_bs = ca_ff_[ca_ff_['restaurant_name'] == 'Burger Station']

#First part of grouping 

agg_funcs = {
    'menu_item_price': [mean_non_zero, median_non_zero, std_non_zero],  # calculate the average, median, and standard dev PRICE
    'restaurant_rating': 'mean', # calculate the average RATING 
    'menu_item' : 'count',
    'number_of_ratings': 'first'
}

grouped_bs = ca_ff_bs.groupby(['restaurant_name','restaurant_location']).agg(agg_funcs).reset_index()
grouped_bs.columns = [' '.join(col).strip() for col in grouped_bs.columns.values]

In [14]:
#Second part of grouping 
bs_lst = ['cheese burger', 'earth burger', 'french fries']

# Filter rows where 'menu_item' contains any item in mcd_lst
menu_items_bs = ca_ff_bs[ca_ff_bs['menu_item'].isin(bs_lst)].sort_values('menu_item')
menu_items_bs = menu_items_bs.drop_duplicates(subset=['restaurant_name', 'restaurant_location', 'menu_item'])

grouped_bs_2 = menu_items_bs.groupby(['restaurant_name', 'restaurant_location'])['menu_item_price'].agg(price_list).reset_index()

grouped_bs_2[['cheeseburger', 'hamburger', 'fries']] = grouped_bs_2['menu_item_price'].apply(pd.Series)
grouped_bs_2.drop(columns=['menu_item_price'], inplace=True)

#Merging the grouped dfs together 
merged_bs = pd.merge(grouped_bs, grouped_bs_2, on=['restaurant_name', 'restaurant_location'], how='inner')
merged_bs['combo'] = np.nan
merged_bs['specialty_item'] = np.nan


merged_bs

,restaurant_name,restaurant_location,menu_item_price mean_non_zero,menu_item_price median_non_zero,menu_item_price std_non_zero,restaurant_rating mean,menu_item count,number_of_ratings first,cheeseburger,hamburger,fries,combo,specialty_item
0,Burger Station,"10361 rockingham drive, sacramento, ca, 95827, us",11.260423,12.93,3.974134,0.0,71,None,4.48,NaN,NaN,NaN,NaN
1,Burger Station,"603 saratoga avenue, suite 30, san jose, ca, 9...",11.672796,11.99,5.526955,4.8,93,450+,12.99,11.99,6.99,NaN,NaN


In [15]:
#'Super Burger'

ca_ff_sb = ca_ff_[ca_ff_['restaurant_name'] == 'Super Burger']

#First part of grouping 

agg_funcs = {
    'menu_item_price': [mean_non_zero, median_non_zero, std_non_zero],  # calculate the average, median, and standard dev PRICE
    'restaurant_rating': 'mean', # calculate the average RATING 
    'menu_item' : 'count',
    'number_of_ratings': 'first'
}

grouped_sb = ca_ff_sb.groupby(['restaurant_name','restaurant_location']).agg(agg_funcs).reset_index()
grouped_sb.columns = [' '.join(col).strip() for col in grouped_sb.columns.values]

In [16]:
#Second part of grouping 
sb_lst = ['fries']

# Filter rows where 'menu_item' contains any item in mcd_lst
menu_items_sb = ca_ff_sb[ca_ff_sb['menu_item'].isin(sb_lst)].sort_values('menu_item')
menu_items_sb = menu_items_sb.drop_duplicates(subset=['restaurant_name', 'restaurant_location', 'menu_item'])

grouped_sb_2 = menu_items_sb.groupby(['restaurant_name', 'restaurant_location'])['menu_item_price'].agg(price_list).reset_index()

grouped_sb_2[['fries']] = grouped_sb_2['menu_item_price'].apply(pd.Series)
grouped_sb_2.drop(columns=['menu_item_price'], inplace=True)

#Merging the grouped dfs together 
merged_sb = pd.merge(grouped_sb, grouped_sb_2, on=['restaurant_name', 'restaurant_location'], how='inner')
merged_sb['combo'] = np.nan
merged_sb['specialty_item'] = np.nan
merged_sb['cheeseburger'] = np.nan
merged_sb['hamburger'] = np.nan


merged_sb

,restaurant_name,restaurant_location,menu_item_price mean_non_zero,menu_item_price median_non_zero,menu_item_price std_non_zero,restaurant_rating mean,menu_item count,number_of_ratings first,fries,combo,specialty_item,cheeseburger,hamburger
0,Super Burger,"1132, clovis, ca, 93612, us",9.347534,9.75,5.030100,4.7,73,130+,3.99,NaN,NaN,NaN,NaN
1,Super Burger,"2505 3rd st, ceres, ca, 95307, us",5.388305,4.99,2.466143,0.0,59,None,2.09,NaN,NaN,NaN,NaN
2,Super Burger,"5831 s santa fe ave, vernon, ca, 90058, us",11.331023,11.99,3.970926,4.8,440,280+,4.99,NaN,NaN,NaN,NaN


In [17]:
#'Burger Basket'

ca_ff_basket = ca_ff_[ca_ff_['restaurant_name'] == 'Burger Basket']

#First part of grouping 

agg_funcs = {
    'menu_item_price': [mean_non_zero, median_non_zero, std_non_zero],  # calculate the average, median, and standard dev PRICE
    'restaurant_rating': 'mean', # calculate the average RATING 
    'menu_item' : 'count',
    'number_of_ratings': 'first'
}

grouped_basket = ca_ff_basket.groupby(['restaurant_name','restaurant_location']).agg(agg_funcs).reset_index()
grouped_basket.columns = [' '.join(col).strip() for col in grouped_basket.columns.values]

In [18]:
#Second part of grouping 
basket_lst = ['cheeseburger','french fries', 'hamburger']

# Filter rows where 'menu_item' contains any item in mcd_lst
menu_items_basket = ca_ff_basket[ca_ff_basket['menu_item'].isin(basket_lst)].sort_values('menu_item')
menu_items_basket = menu_items_basket.drop_duplicates(subset=['restaurant_name', 'restaurant_location', 'menu_item'])

grouped_basket_2 = menu_items_basket.groupby(['restaurant_name', 'restaurant_location'])['menu_item_price'].agg(price_list).reset_index()

grouped_basket_2[['cheeseburger','fries', 'hamburger']] = grouped_basket_2['menu_item_price'].apply(pd.Series)
grouped_basket_2.drop(columns=['menu_item_price'], inplace=True)

#Merging the grouped dfs together 
merged_basket = pd.merge(grouped_basket, grouped_basket_2, on=['restaurant_name', 'restaurant_location'], how='inner')
merged_basket['combo'] = np.nan
merged_basket['specialty_item'] = np.nan
#merged_basket['cheeseburger'] = np.nan

merged_basket

,restaurant_name,restaurant_location,menu_item_price mean_non_zero,menu_item_price median_non_zero,menu_item_price std_non_zero,restaurant_rating mean,menu_item count,number_of_ratings first,cheeseburger,fries,hamburger,combo,specialty_item
0,Burger Basket,"12155 firestone blvd, norwalk, ca, 90650, us",9.542857,8.99,6.167034,4.6,630,220+,7.5,3.9,6.75,NaN,NaN
1,Burger Basket,"501 west duarte road, monrovia, ca, 91016, us",7.210571,6.60,4.622437,4.8,385,500+,6.0,4.5,5.35,NaN,NaN


In [19]:
#'Burger Factory'

ca_ff_factory = ca_ff_[ca_ff_['restaurant_name'] == 'Burger Factory']

#First part of grouping 

agg_funcs = {
    'menu_item_price': [mean_non_zero, median_non_zero, std_non_zero],  # calculate the average, median, and standard dev PRICE
    'restaurant_rating': 'mean', # calculate the average RATING 
    'menu_item' : 'count',
    'number_of_ratings': 'first'
}

grouped_factory = ca_ff_factory.groupby(['restaurant_name','restaurant_location']).agg(agg_funcs).reset_index()
grouped_factory.columns = [' '.join(col).strip() for col in grouped_factory.columns.values]

In [20]:
#Second part of grouping 
factory_lst = ['colossal burger']

# Filter rows where 'menu_item' contains any item in mcd_lst
menu_items_factory = ca_ff_factory[ca_ff_factory['menu_item'].isin(factory_lst)].sort_values('menu_item')
menu_items_factory = menu_items_factory.drop_duplicates(subset=['restaurant_name', 'restaurant_location', 'menu_item'])

grouped_factory_2 = menu_items_factory.groupby(['restaurant_name', 'restaurant_location'])['menu_item_price'].agg(price_list).reset_index()

grouped_factory_2[['hamburger']] = grouped_factory_2['menu_item_price'].apply(pd.Series)
grouped_factory_2.drop(columns=['menu_item_price'], inplace=True)

#Merging the grouped dfs together 
merged_factory = pd.merge(grouped_factory, grouped_factory_2, on=['restaurant_name', 'restaurant_location'], how='inner')
merged_factory['combo'] = np.nan
merged_factory['specialty_item'] = np.nan
merged_factory['cheeseburger'] = np.nan
merged_factory['fries'] = np.nan


merged_factory

,restaurant_name,restaurant_location,menu_item_price mean_non_zero,menu_item_price median_non_zero,menu_item_price std_non_zero,restaurant_rating mean,menu_item count,number_of_ratings first,hamburger,combo,specialty_item,cheeseburger,fries
0,Burger Factory,"1358 w martin luther king jr blvd, los angeles...",11.99954,12.5,6.786709,4.5,435,700+,9.0,NaN,NaN,NaN,NaN


In [21]:
#'Cypress Best Burgers'

ca_ff_cypress = ca_ff_[ca_ff_['restaurant_name'] == 'Cypress Best Burgers']

#First part of grouping 

agg_funcs = {
    'menu_item_price': [mean_non_zero, median_non_zero, std_non_zero],  # calculate the average, median, and standard dev PRICE
    'restaurant_rating': 'mean', # calculate the average RATING 
    'menu_item' : 'count',
    'number_of_ratings': 'first'
}

grouped_cypress = ca_ff_cypress.groupby(['restaurant_name','restaurant_location']).agg(agg_funcs).reset_index()
grouped_cypress.columns = [' '.join(col).strip() for col in grouped_cypress.columns.values]

In [22]:
#Second part of grouping 
cypress_lst = ['14 lb burger', '14 lb cheeseburger','french fries']

# Filter rows where 'menu_item' contains any item in mcd_lst
menu_items_cypress = ca_ff_cypress[ca_ff_cypress['menu_item'].isin(cypress_lst)].sort_values('menu_item')
menu_items_cypress = menu_items_cypress.drop_duplicates(subset=['restaurant_name', 'restaurant_location', 'menu_item'])

grouped_cypress_2 = menu_items_cypress.groupby(['restaurant_name', 'restaurant_location'])['menu_item_price'].agg(price_list).reset_index()

grouped_cypress_2[['hamburger', 'cheeseburger', 'fries']] = grouped_cypress_2['menu_item_price'].apply(pd.Series)
grouped_cypress_2.drop(columns=['menu_item_price'], inplace=True)

#Merging the grouped dfs together 
merged_cypress = pd.merge(grouped_cypress, grouped_cypress_2, on=['restaurant_name', 'restaurant_location'], how='inner')
merged_cypress['combo'] = np.nan
merged_cypress['specialty_item'] = np.nan


merged_cypress

,restaurant_name,restaurant_location,menu_item_price mean_non_zero,menu_item_price median_non_zero,menu_item_price std_non_zero,restaurant_rating mean,menu_item count,number_of_ratings first,hamburger,cheeseburger,fries,combo,specialty_item
0,Cypress Best Burgers,"3240 n figueroa st, los angeles, ca, 90065, us",11.7519,12.685,4.825941,4.8,600,1,6.79,7.79,6.29,NaN,NaN


In [23]:
#'Gold Star Hamburgers'

ca_ff_gold = ca_ff_[ca_ff_['restaurant_name'] == 'Gold Star Hamburgers']

#First part of grouping 

agg_funcs = {
    'menu_item_price': [mean_non_zero, median_non_zero, std_non_zero],  # calculate the average, median, and standard dev PRICE
    'restaurant_rating': 'mean', # calculate the average RATING 
    'menu_item' : 'count',
    'number_of_ratings': 'first'
}

grouped_gold = ca_ff_gold.groupby(['restaurant_name','restaurant_location']).agg(agg_funcs).reset_index()
grouped_gold.columns = [' '.join(col).strip() for col in grouped_gold.columns.values]

In [24]:
#Second part of grouping 
gold_lst = ['cheeseburger', 'fries','hamburger']

# Filter rows where 'menu_item' contains any item in mcd_lst
menu_items_gold = ca_ff_gold[ca_ff_gold['menu_item'].isin(gold_lst)].sort_values('menu_item')
menu_items_gold = menu_items_gold.drop_duplicates(subset=['restaurant_name', 'restaurant_location', 'menu_item'])

grouped_gold_2 = menu_items_gold.groupby(['restaurant_name', 'restaurant_location'])['menu_item_price'].agg(price_list).reset_index()

grouped_gold_2[['cheeseburger', 'fries', 'hamburger']] = grouped_gold_2['menu_item_price'].apply(pd.Series)
grouped_gold_2.drop(columns=['menu_item_price'], inplace=True)

#Merging the grouped dfs together 
merged_gold = pd.merge(grouped_gold, grouped_gold_2, on=['restaurant_name', 'restaurant_location'], how='inner')
merged_gold['combo'] = np.nan
merged_gold['specialty_item'] = np.nan


merged_gold

,restaurant_name,restaurant_location,menu_item_price mean_non_zero,menu_item_price median_non_zero,menu_item_price std_non_zero,restaurant_rating mean,menu_item count,number_of_ratings first,cheeseburger,fries,hamburger,combo,specialty_item
0,Gold Star Hamburgers,"1623 s brand blvd, glendale, ca, 91204, us",10.537213,10.69,5.271905,4.8,305,600+,8.29,5.25,7.89,NaN,NaN


In [25]:
#'Hook Burger'

ca_ff_hook = ca_ff_[ca_ff_['restaurant_name'] == 'Hook Burger']

#First part of grouping 

agg_funcs = {
    'menu_item_price': [mean_non_zero, median_non_zero, std_non_zero],  # calculate the average, median, and standard dev PRICE
    'restaurant_rating': 'mean', # calculate the average RATING 
    'menu_item' : 'count',
    'number_of_ratings': 'first'
}

grouped_hook = ca_ff_hook.groupby(['restaurant_name','restaurant_location']).agg(agg_funcs).reset_index()
grouped_hook.columns = [' '.join(col).strip() for col in grouped_hook.columns.values]

In [26]:
#Second part of grouping 
hook_lst = ['french fries','hook burger']

# Filter rows where 'menu_item' contains any item in mcd_lst
menu_items_hook = ca_ff_hook[ca_ff_hook['menu_item'].isin(hook_lst)].sort_values('menu_item')
menu_items_hook = menu_items_hook.drop_duplicates(subset=['restaurant_name', 'restaurant_location', 'menu_item'])

grouped_hook_2 = menu_items_hook.groupby(['restaurant_name', 'restaurant_location'])['menu_item_price'].agg(price_list).reset_index()

grouped_hook_2[['fries', 'hamburger']] = grouped_hook_2['menu_item_price'].apply(pd.Series)
grouped_hook_2.drop(columns=['menu_item_price'], inplace=True)

#Merging the grouped dfs together 
merged_hook = pd.merge(grouped_hook, grouped_hook_2, on=['restaurant_name', 'restaurant_location'], how='inner')
merged_hook['combo'] = np.nan
merged_hook['specialty_item'] = np.nan
merged_hook['cheeseburger'] = np.nan

merged_gold

,restaurant_name,restaurant_location,menu_item_price mean_non_zero,menu_item_price median_non_zero,menu_item_price std_non_zero,restaurant_rating mean,menu_item count,number_of_ratings first,cheeseburger,fries,hamburger,combo,specialty_item
0,Gold Star Hamburgers,"1623 s brand blvd, glendale, ca, 91204, us",10.537213,10.69,5.271905,4.8,305,600+,8.29,5.25,7.89,NaN,NaN


In [27]:
#'Burger Palace'

ca_ff_palace = ca_ff_[ca_ff_['restaurant_name'] == 'Burger Palace']

#First part of grouping 

agg_funcs = {
    'menu_item_price': [mean_non_zero, median_non_zero, std_non_zero],  # calculate the average, median, and standard dev PRICE
    'restaurant_rating': 'mean', # calculate the average RATING 
    'menu_item' : 'count',
    'number_of_ratings': 'first'
}

grouped_palace = ca_ff_palace.groupby(['restaurant_name','restaurant_location']).agg(agg_funcs).reset_index()
grouped_palace.columns = [' '.join(col).strip() for col in grouped_palace.columns.values]

In [28]:
#Second part of grouping 
palace_lst = ['14 lb hamburger', 'cheeseburger', 'small french fries']

# Filter rows where 'menu_item' contains any item in mcd_lst
menu_items_palace = ca_ff_palace[ca_ff_palace['menu_item'].isin(palace_lst)].sort_values('menu_item')
menu_items_palace = menu_items_palace.drop_duplicates(subset=['restaurant_name', 'restaurant_location', 'menu_item'])

grouped_palace_2 = menu_items_palace.groupby(['restaurant_name', 'restaurant_location'])['menu_item_price'].agg(price_list).reset_index()

grouped_palace_2[['hamburger', 'cheeseburger', 'fries']] = grouped_palace_2['menu_item_price'].apply(pd.Series)
grouped_palace_2.drop(columns=['menu_item_price'], inplace=True)

#Merging the grouped dfs together 
merged_palace = pd.merge(grouped_palace, grouped_palace_2, on=['restaurant_name', 'restaurant_location'], how='inner')
merged_palace['combo'] = np.nan
merged_palace['specialty_item'] = np.nan

merged_palace

,restaurant_name,restaurant_location,menu_item_price mean_non_zero,menu_item_price median_non_zero,menu_item_price std_non_zero,restaurant_rating mean,menu_item count,number_of_ratings first,hamburger,cheeseburger,fries,combo,specialty_item
0,Burger Palace,"100 e manchester ave, los angeles, ca, 90003, us",8.719149,8.05,4.982336,4.5,376,420+,5.2,5.75,3.3,NaN,NaN
1,Burger Palace,"7126 s western ave, los angeles, ca, 90047, us",9.051648,8.05,5.744593,4.7,182,800+,5.2,5.75,3.3,NaN,NaN


In [29]:
#'Varsity Burgers'

ca_ff_varsity = ca_ff_[ca_ff_['restaurant_name'] == 'Varsity Burgers']

#First part of grouping 

agg_funcs = {
    'menu_item_price': [mean_non_zero, median_non_zero, std_non_zero],  # calculate the average, median, and standard dev PRICE
    'restaurant_rating': 'mean', # calculate the average RATING 
    'menu_item' : 'count',
    'number_of_ratings': 'first'
}

grouped_varsity = ca_ff_varsity.groupby(['restaurant_name','restaurant_location']).agg(agg_funcs).reset_index()
grouped_varsity.columns = [' '.join(col).strip() for col in grouped_varsity.columns.values]

In [30]:
#Second part of grouping 
varsity_lst = ['cheeseburger', 'french fries', 'hamburger']

# Filter rows where 'menu_item' contains any item in mcd_lst
menu_items_varsity = ca_ff_varsity[ca_ff_varsity['menu_item'].isin(varsity_lst)].sort_values('menu_item')
menu_items_varsity = menu_items_varsity.drop_duplicates(subset=['restaurant_name', 'restaurant_location', 'menu_item'])

grouped_varsity_2 = menu_items_varsity.groupby(['restaurant_name', 'restaurant_location'])['menu_item_price'].agg(price_list).reset_index()

grouped_varsity_2[['cheeseburger', 'fries','hamburger']] = grouped_varsity_2['menu_item_price'].apply(pd.Series)
grouped_varsity_2.drop(columns=['menu_item_price'], inplace=True)

#Merging the grouped dfs together 
merged_varsity = pd.merge(grouped_varsity, grouped_varsity_2, on=['restaurant_name', 'restaurant_location'], how='inner')
merged_varsity['combo'] = np.nan
merged_varsity['specialty_item'] = np.nan

merged_varsity

,restaurant_name,restaurant_location,menu_item_price mean_non_zero,menu_item_price median_non_zero,menu_item_price std_non_zero,restaurant_rating mean,menu_item count,number_of_ratings first,cheeseburger,fries,hamburger,combo,specialty_item
0,Varsity Burgers,"600 w lincoln ave, anaheim, ca, 92805, us",10.043836,9.35,2.468194,4.8,584,3,7.85,5.8,7.35,NaN,NaN


In [31]:
#'Burger Boss'

ca_ff_boss = ca_ff_[ca_ff_['restaurant_name'] == 'Burger Boss']

#First part of grouping 

agg_funcs = {
    'menu_item_price': [mean_non_zero, median_non_zero, std_non_zero],  # calculate the average, median, and standard dev PRICE
    'restaurant_rating': 'mean', # calculate the average RATING 
    'menu_item' : 'count',
    'number_of_ratings': 'first'
}

grouped_boss = ca_ff_boss.groupby(['restaurant_name','restaurant_location']).agg(agg_funcs).reset_index()
grouped_boss.columns = [' '.join(col).strip() for col in grouped_boss.columns.values]

In [32]:
#Second part of grouping 
boss_lst = ['beef single burger', 'fresh cut boss fries','original boss burger']

# Filter rows where 'menu_item' contains any item in mcd_lst
menu_items_boss = ca_ff_boss[ca_ff_boss['menu_item'].isin(boss_lst)].sort_values('menu_item')
menu_items_boss = menu_items_boss.drop_duplicates(subset=['restaurant_name', 'restaurant_location', 'menu_item'])

grouped_boss_2 = menu_items_boss.groupby(['restaurant_name', 'restaurant_location'])['menu_item_price'].agg(price_list).reset_index()

grouped_boss_2[['hamburger', 'fries','cheeseburger']] = grouped_boss_2['menu_item_price'].apply(pd.Series)
grouped_boss_2.drop(columns=['menu_item_price'], inplace=True)

#Merging the grouped dfs together 
merged_boss = pd.merge(grouped_boss, grouped_boss_2, on=['restaurant_name', 'restaurant_location'], how='inner')
merged_boss['combo'] = np.nan
merged_boss['specialty_item'] = np.nan

merged_boss

,restaurant_name,restaurant_location,menu_item_price mean_non_zero,menu_item_price median_non_zero,menu_item_price std_non_zero,restaurant_rating mean,menu_item count,number_of_ratings first,hamburger,fries,cheeseburger,combo,specialty_item
0,Burger Boss,"15080 kensington park drive, tustin, ca, 92780...",10.709259,11.0,3.923081,4.7,28,490+,11.0,9.0,13.0,NaN,NaN
1,Burger Boss,"27690-b santa margarita pkwy, mission viejo, c...",10.709259,11.0,3.923081,4.7,28,500+,11.0,9.0,13.0,NaN,NaN


In [33]:
#'P & G Burgers'

ca_ff_pg = ca_ff_[ca_ff_['restaurant_name'] == 'P & G Burgers']

#First part of grouping 

agg_funcs = {
    'menu_item_price': [mean_non_zero, median_non_zero, std_non_zero],  # calculate the average, median, and standard dev PRICE
    'restaurant_rating': 'mean', # calculate the average RATING 
    'menu_item' : 'count',
    'number_of_ratings': 'first'
}

grouped_pg = ca_ff_pg.groupby(['restaurant_name','restaurant_location']).agg(agg_funcs).reset_index()
grouped_pg.columns = [' '.join(col).strip() for col in grouped_pg.columns.values]

In [34]:
#Second part of grouping 
pg_lst = ['burger 14 lb', 'cheeseburger 14 lb', 'french fries']

# Filter rows where 'menu_item' contains any item in mcd_lst
menu_items_pg = ca_ff_pg[ca_ff_pg['menu_item'].isin(pg_lst)].sort_values('menu_item')
menu_items_pg = menu_items_pg.drop_duplicates(subset=['restaurant_name', 'restaurant_location', 'menu_item'])

grouped_pg_2 = menu_items_pg.groupby(['restaurant_name', 'restaurant_location'])['menu_item_price'].agg(price_list).reset_index()

grouped_pg_2[['hamburger', 'cheeseburger','fries']] = grouped_pg_2['menu_item_price'].apply(pd.Series)
grouped_pg_2.drop(columns=['menu_item_price'], inplace=True)

#Merging the grouped dfs together 
merged_pg = pd.merge(grouped_pg, grouped_pg_2, on=['restaurant_name', 'restaurant_location'], how='inner')
merged_pg['combo'] = np.nan
merged_pg['specialty_item'] = np.nan

merged_pg

,restaurant_name,restaurant_location,menu_item_price mean_non_zero,menu_item_price median_non_zero,menu_item_price std_non_zero,restaurant_rating mean,menu_item count,number_of_ratings first,hamburger,cheeseburger,fries,combo,specialty_item
0,P & G Burgers,"190 west valley boulevard, colton, ca, 92324, us",11.068,10.79,5.471006,4.6,696,2,5.99,6.99,6.99,NaN,NaN


In [35]:
#'R Burgers'

ca_ff_r = ca_ff_[ca_ff_['restaurant_name'] == 'R Burgers']

#First part of grouping 

agg_funcs = {
    'menu_item_price': [mean_non_zero, median_non_zero, std_non_zero],  # calculate the average, median, and standard dev PRICE
    'restaurant_rating': 'mean', # calculate the average RATING 
    'menu_item' : 'count',
    'number_of_ratings': 'first'
}

grouped_r = ca_ff_r.groupby(['restaurant_name','restaurant_location']).agg(agg_funcs).reset_index()
grouped_r.columns = [' '.join(col).strip() for col in grouped_r.columns.values]

In [36]:
#Second part of grouping 
r_lst = ['6 double cheeseburger', 'r burger']

# Filter rows where 'menu_item' contains any item in mcd_lst
menu_items_r = ca_ff_r[ca_ff_r['menu_item'].isin(r_lst)].sort_values('menu_item')
menu_items_r = menu_items_r.drop_duplicates(subset=['restaurant_name', 'restaurant_location', 'menu_item'])

grouped_r_2 = menu_items_r.groupby(['restaurant_name', 'restaurant_location'])['menu_item_price'].agg(price_list).reset_index()

grouped_r_2[['cheeseburger', 'hamburger']] = grouped_r_2['menu_item_price'].apply(pd.Series)
grouped_r_2.drop(columns=['menu_item_price'], inplace=True)

#Merging the grouped dfs together 
merged_r = pd.merge(grouped_r, grouped_r_2, on=['restaurant_name', 'restaurant_location'], how='inner')
merged_r['combo'] = np.nan
merged_r['specialty_item'] = np.nan
merged_r['fries'] = np.nan

merged_r

,restaurant_name,restaurant_location,menu_item_price mean_non_zero,menu_item_price median_non_zero,menu_item_price std_non_zero,restaurant_rating mean,menu_item count,number_of_ratings first,cheeseburger,hamburger,combo,specialty_item,fries
0,R Burgers,"1666 university avenue, riverside, ca, 92507, us",13.667875,13.49,5.150436,4.6,243,410+,19.49,11.99,NaN,NaN,NaN
1,R Burgers,"23750 alessandro boulevard suite b, moreno val...",11.968382,12.72,4.839656,4.7,138,250+,17.99,10.99,NaN,NaN,NaN
2,R Burgers,"4890 south motor lane, ontario, ca, 91761, us",13.685625,13.47,5.147740,4.6,243,210+,19.49,11.99,NaN,NaN,NaN
3,R Burgers,"5980 van buren blvd, riverside, ca, 92503, us",13.684875,13.49,5.141850,4.6,243,150+,19.49,11.99,NaN,NaN,NaN
4,R Burgers,"6231 valley springs pkwy, riverside, ca, 92507...",13.676625,13.49,5.146472,4.5,243,68,19.49,11.99,NaN,NaN,NaN


In [37]:
grouped_r_2

,restaurant_name,restaurant_location,cheeseburger,hamburger
0,R Burgers,"1666 university avenue, riverside, ca, 92507, us",19.49,11.99
1,R Burgers,"23750 alessandro boulevard suite b, moreno val...",17.99,10.99
2,R Burgers,"4890 south motor lane, ontario, ca, 91761, us",19.49,11.99
3,R Burgers,"5980 van buren blvd, riverside, ca, 92503, us",19.49,11.99
4,R Burgers,"6231 valley springs pkwy, riverside, ca, 92507...",19.49,11.99


In [38]:
#'Betty Burgers'

ca_ff_betty = ca_ff_[ca_ff_['restaurant_name'] == 'Betty Burgers']

#First part of grouping 

agg_funcs = {
    'menu_item_price': [mean_non_zero, median_non_zero, std_non_zero],  # calculate the average, median, and standard dev PRICE
    'restaurant_rating': 'mean', # calculate the average RATING 
    'menu_item' : 'count',
    'number_of_ratings': 'first'
}

grouped_betty = ca_ff_betty.groupby(['restaurant_name','restaurant_location']).agg(agg_funcs).reset_index()
grouped_betty.columns = [' '.join(col).strip() for col in grouped_betty.columns.values]

In [39]:
#Second part of grouping 
betty_lst = ['basic betty', 'crispy fries  regular']

# Filter rows where 'menu_item' contains any item in mcd_lst
menu_items_betty = ca_ff_betty[ca_ff_betty['menu_item'].isin(betty_lst)].sort_values('menu_item')
menu_items_betty = menu_items_betty.drop_duplicates(subset=['restaurant_name', 'restaurant_location', 'menu_item'])

grouped_betty_2 = menu_items_betty.groupby(['restaurant_name', 'restaurant_location'])['menu_item_price'].agg(price_list).reset_index()

grouped_betty_2[['hamburger', 'fries']] = grouped_betty_2['menu_item_price'].apply(pd.Series)
grouped_betty_2.drop(columns=['menu_item_price'], inplace=True)

#Merging the grouped dfs together 
merged_betty = pd.merge(grouped_betty, grouped_betty_2, on=['restaurant_name', 'restaurant_location'], how='inner')
merged_betty['combo'] = np.nan
merged_betty['specialty_item'] = np.nan
merged_betty['cheeseburger'] = np.nan

merged_betty

,restaurant_name,restaurant_location,menu_item_price mean_non_zero,menu_item_price median_non_zero,menu_item_price std_non_zero,restaurant_rating mean,menu_item count,number_of_ratings first,hamburger,fries,combo,specialty_item,cheeseburger
0,Betty Burgers,"415 trout gulch road, aptos, ca, 95003, us",10.079400,8.99,5.014637,4.7,116,28,13.49,3.99,NaN,NaN,NaN
1,Betty Burgers,"505 seabright ave, santa cruz, ca, 95062, us",8.660145,7.99,5.298367,4.5,69,49,13.49,3.99,NaN,NaN,NaN


In [40]:
ubereats_fflocal_prices = pd.concat(
    [
        merged_bm, 
        merged_bs, 
        merged_sb, 
        merged_basket, 
        merged_factory, 
        merged_cypress, 
        merged_gold, 
        merged_hook,
        merged_palace,
        merged_varsity,
        merged_boss,
        merged_pg,
        merged_r,
        merged_betty
    ]
).reset_index(drop=True)

In [109]:
ubereats_fflocal_prices = merged_bj

In [41]:
pattern = r",\s*([a-zA-Z]{2})\s*,?\s*(\d{5}(?:-\d{4})?)"

def extract_state_zip(address):
    match = re.search(pattern, address)
    if match:
        state, zip_code = match.groups()
        return state, zip_code
    else:
        return None, None

# Apply the function to extract state and zip code

ubereats_fflocal_prices[['state', 'zip']] = ubereats_fflocal_prices['restaurant_location'].apply(lambda x: pd.Series(extract_state_zip(x)))

ubereats_fflocal_prices['zip'] = ubereats_fflocal_prices['zip'].str.split('-').str[0].astype(int)

#Get county 
ubereats_fflocal_prices = ubereats_fflocal_prices.merge(ca_zip_count, on = 'zip')

In [42]:
specific_date = datetime.strptime('05272024', '%m%d%Y')
# Assign the datetime object to the entire 'date' column
ubereats_fflocal_prices['date'] = specific_date
ubereats_fflocal_prices['uber_eats'] = 1
ubereats_fflocal_prices['post_policy'] = 1
ubereats_fflocal_prices['fast_food'] = 1
ubereats_fflocal_prices['local'] = 1

In [43]:
ubereats_fflocal_prices.to_csv('final_processed_prices_ubereats_ca_fflocal05272024.csv', index = True)